In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
import tensorflow as tf
import tensorflow_hub as hub
import json

[nltk_data] Downloading package punkt to /home/mxhow/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/mxhow/anaconda3/envs/hackathon1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mxhow/anaconda3/envs/hackathon1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mxhow/anaconda3/envs/hackathon1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood a

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(module_url)

In [4]:
with open("pros_cons.json", "r") as file:
    json_file = json.load(file)

In [8]:
json_file['apple-iphone-11']['cons'][0]

['Cameras lack optical zoom, Some may not like the thicker bezels, Comes with a 5-watt charger',
 "No big increases in wireless performance. Next year's iPhone will likely mark a major change.",
 "Only Pro models get the 2x telephoto. The ultrawide-angle camera doesn’t add Night Mode. No USB-C port. The Pro phones have a faster 18-watt charger but iPhone 11 doesn't. Still has a good (but not OLED) display.",
 'no expandable storage, weak location accuracy, moderate audio performance, weak power supply, only 12 months warranty, restricted NFC',
 'Starting storage of 64GB',
 'No OLED display, Bezels are still large, No zoom lens',
 'iOS 13 is buggy, Just one fairly large size, Get out of here with this 5W charger, The garden is beautiful, but the walls grow ever higher',
 "The battery life and display aren't as nice as the Pro. It definitely feels cheaper.",
 'No zoom lens on camera, LCD screen not as good as OLED for movies, Only comes with 5W charger']

In [25]:
cons_list = []
for x in json_file['apple-iphone-11']['cons'][0]:
    spec = x.split(",")
    cleaned = [x.strip() for x in spec]
    cons_list += cleaned
cons_list2 = []
for y in cons_list:
    if '.' in y:
        spec = y.split(".")
        cleaned = [x.strip() for x in spec]
        cons_list2 += cleaned
    else:
        cons_list2.append(y)

In [27]:
pros_list = []
for x in json_file['apple-iphone-11']['pros'][0]:
    spec = x.split(",")
    cleaned = [x.strip() for x in spec]
    pros_list += cleaned
pros_list2 = []
for y in pros_list:
    if '.' in y:
        spec = y.split(".")
        cleaned = [x.strip() for x in spec]
        pros_list2 += cleaned
    else:
        pros_list2.append(y)

In [26]:
cons_list2

['Cameras lack optical zoom',
 'Some may not like the thicker bezels',
 'Comes with a 5-watt charger',
 'No big increases in wireless performance',
 "Next year's iPhone will likely mark a major change",
 '',
 'Only Pro models get the 2x telephoto',
 'The ultrawide-angle camera doesn’t add Night Mode',
 'No USB-C port',
 "The Pro phones have a faster 18-watt charger but iPhone 11 doesn't",
 'Still has a good (but not OLED) display',
 '',
 'no expandable storage',
 'weak location accuracy',
 'moderate audio performance',
 'weak power supply',
 'only 12 months warranty',
 'restricted NFC',
 'Starting storage of 64GB',
 'No OLED display',
 'Bezels are still large',
 'No zoom lens',
 'iOS 13 is buggy',
 'Just one fairly large size',
 'Get out of here with this 5W charger',
 'The garden is beautiful',
 'but the walls grow ever higher',
 "The battery life and display aren't as nice as the Pro",
 'It definitely feels cheaper',
 '',
 'No zoom lens on camera',
 'LCD screen not as good as OLED fo

In [28]:
pros_list2

['Impressive speed',
 'New wide and ultra wide cameras',
 "Elegant design that doesn't make sacrifices for a lower price",
 'Reasonable price',
 'New wide-angle camera option',
 'Fastest processor available',
 'Accurate display colors',
 '',
 'Even faster speed',
 'improved battery life',
 "The iPhone 11's cameras get an excellent new Night Mode and an ultrawide-angle camera that can add extra detail in photos",
 'Fantastic video camera',
 '',
 'powerful SoC',
 'bright and colour accurate LCD panel',
 'dust and waterproof',
 'wide LTE frequency support',
 'wireless charging',
 'good battery life',
 'Fast performance',
 'New camera features',
 'Strong battery life',
 'Tougher and water resistant design',
 'glossy back',
 '',
 'The screen has excellent brightness and contrast',
 '',
 'Apple A13 Bionic is the fastest smartphone chip on the planet',
 '',
 'Excellent battery life',
 'fast charging support',
 '',
 'Stereo speakers with great quality',
 '',
 'iOS provides smooth user experien

In [40]:
# def remove_rubbish(sentence):
#     replacedstr = re.sub('\n+', ' ', sentence)
#     return replacedstr

In [41]:
# df = pd.read_csv("test.csv",encoding='latin')

In [59]:
# from nltk.tokenize import sent_tokenize
# sentences = []
# for s in json_file:
#     sentences.append(sent_tokenize(s))

# sentences = [y for x in sentences for y in x] # flatten list

In [29]:
sentences = cons_list2

In [30]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(sentences))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [31]:
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(message_embeddings[i].reshape(1,512), message_embeddings[j].reshape(1,512))[0,0]

In [33]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [34]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [35]:
# Extract top 10 sentences as the summary
for i in range(5):
    print(ranked_sentences[i][1])

Still has a good (but not OLED) display
The battery life and display aren't as nice as the Pro
no expandable storage
No OLED display
Bezels are still large
